# Project 3 - Part 2 Extract from TMDB 

Source: https://www.themoviedb.org/

In [24]:
#login with api credentials for tmbd
import json
with open('C:\\Users\\nena_\\.secret\\tmbd_api.json','r') as f:
    login = json.load(f)
#display the keys of the loaded dict 
login.keys()

dict_keys(['api key', 'api read token'])

In [25]:
#import tmdb package and set the api_key variable to the API key 
import tmdbsimple as tmdb 
tmdb.API_KEY = login['api key']

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm_notebook

import os, time,json
import tmdbsimple as tmdb 
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)



['.ipynb_checkpoints',
 'combined_api_results.csv.gz',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'final_tmdb_data_2010.csv.gz',
 'final_tmdb_data_2011.csv.gz',
 'final_tmdb_data_2012.csv.gz',
 'final_tmdb_data_2013.csv.gz',
 'final_tmdb_data_2014.csv.gz',
 'final_tmdb_data_2015.csv.gz',
 'final_tmdb_data_2018.csv.gz',
 'final_tmdb_data_2019.csv.gz',
 'p4_combined_api_results.csv.gz',
 'title-akas-us-only.csv',
 'title.basics.csv.gz',
 'title.ratings.csv.gz',
 'title_basics_combined.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_api_results_2010.json',
 'tmdb_api_results_2011.json',
 'tmdb_api_results_2012.json',
 'tmdb_api_results_2013.json',
 'tmdb_api_results_2014.json',
 'tmdb_api_results_2015.json',
 'tmdb_api_results_2018.json',
 'tmdb_api_results_2019.json']

* Your stakeholder wants you to extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".

### Saving the Movie Certification/MPAA Rating

#### Defining the Function 

In [27]:
def get_movie_with_rating(movie_id):
    
    #get the movie object for the current id 
    movie = tmdb.Movies(movie_id)
    
    #save the .info .releases dictionaries
    info = movie.info()
    
    releases = movie.releases()
    #Loop through countries in releases 
    for c in releases['countries']:
        #if the country abbreviation == US
        if c['iso_3166_1'] == 'US':
            ## Save a certification key in info with the certification 
            info['certification'] = c['certification']
            
    return info 

In [35]:
test = get_movie_with_rating("tt0848228") 
test

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 96.656,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path'

In [29]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


### Read in Title Basics

In [7]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/title.basics.csv.gz')

basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016,NaN,90,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


### Extracting years 2010-2019 

In [42]:
YEARS_TO_GET = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]

In [38]:
errors = [ ]

In [39]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON File to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)


    # If it does not exist: create it
    if file_exists == False:
        
        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            
            json.dump([{'imdb_id':0}],f)

    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    movie_ids

    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)

    previous_df

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]



    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])



    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)



YEARS:   0%|          | 0/1 [00:00<?, ?it/s]

Movies from 2022: 0it [00:00, ?it/s]

In [41]:
#Print message reporting back the number of movie ids that caused an error 
print(f"- Total errors: {len(errors)}")

- Total errors: 0


### Exploratory Data Analysis

In [12]:
# Load in the dataframe from project part 1 as basics:
yr_2010 = pd.read_csv('Data/final_tmdb_data_2010.csv.gz')

yr_2010.head()


,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0312305,0.0,/lqUbt2cy2pnqvxKefbQAtxLS0WA.jpg,NaN,0.0,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",http://www.qqthemovie.com/,23738.0,en,Quantum Quest: A Cassini Space Odyssey,...,0.0,45.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Quantum Quest: A Cassini Space Odyssey,0.0,7.900,8.0,NaN
2,tt0326965,0.0,/xt2klJdKCVGXcoBGQrGfAS0aGDE.jpg,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",http://www.inmysleep.com,40048.0,en,In My Sleep,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Sleepwalking can be deadly.,In My Sleep,0.0,5.318,33.0,PG-13
3,tt0331312,0.0,NaN,NaN,0.0,[],NaN,214026.0,en,This Wretched Life,...,0.0,0.0,[],Released,NaN,This Wretched Life,0.0,5.000,1.0,NaN
4,tt0393049,0.0,/gc9FN5zohhzCt05RkejQIIPLtBl.jpg,NaN,300000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,324352.0,en,Anderson's Cross,...,0.0,98.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Sometimes the boy next door is more than the b...,Anderson's Cross,0.0,4.000,5.0,NaN


In [13]:
# Load in the dataframe from project part 1 as basics:
yr_2011 = pd.read_csv('Data/final_tmdb_data_2011.csv.gz')

yr_2011.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0176694,0.0,/43VrmHOChLcAOG9S5tIFdsxr3u8.jpg,NaN,0.0,"[{'id': 16, 'name': 'Animation'}, {'id': 18, '...",NaN,116369.0,hu,Az ember tragédiája,...,0.0,166.0,"[{'english_name': 'Hungarian', 'iso_639_1': 'h...",Released,NaN,The Tragedy of Man,0.0,7.600,14.0,NaN
2,tt0210470,0.0,NaN,NaN,0.0,"[{'id': 99, 'name': 'Documentary'}]",https://www.leightonpierce.com/cms/index.php?o...,239465.0,en,50 Feet of String,...,0.0,52.0,"[{'english_name': 'No Language', 'iso_639_1': ...",Released,NaN,50 Feet of String,0.0,6.000,3.0,NaN
3,tt0323808,0.0,/oJiBH1aSFv0gOKwb34I7CCKqgRE.jpg,NaN,7750000.0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 27, '...",http://www.thewickertreemovie.com/,79544.0,en,The Wicker Tree,...,0.0,96.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Accept our sacrifice,The Wicker Tree,0.0,3.902,46.0,R
4,tt0372538,0.0,NaN,NaN,5000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",http://www.spythemovie.com/,323867.0,en,Spy,...,0.0,110.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Spy,0.0,5.500,2.0,NaN


In [14]:
# Load in the dataframe from project part 1 as basics:
yr_2012 = pd.read_csv('Data/final_tmdb_data_2012.csv.gz')

yr_2012.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0249516,0.0,/t7zb6CnRQwhzQSq0apR4ESFYiWN.jpg,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 35, '...",http://www.thresholdanimationstudios.com/video...,116977.0,en,Foodfight!,...,73706.0,91.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Foodfight!,0.0,1.923,117.0,PG
2,tt0285252,0.0,/pKOHinAVIkyqLliMaTHCSL1T4zU.jpg,NaN,4000000.0,"[{'id': 35, 'name': 'Comedy'}]",NaN,112074.0,en,Life's a Beach,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Life's a Beach,0.0,2.400,6.0,R
3,tt0293069,0.0,/cj2isIuskSePAwpb2D6yUBZrhkJ.jpg,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}]",NaN,136884.0,en,Dark Blood,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Dark Blood,0.0,6.053,19.0,NaN
4,tt0337692,0.0,/jbcjTR19jatK9Jh4yfpNUXwPGh6.jpg,NaN,25000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",http://www.ontheroad-themovie.com/?lang=en,83770.0,en,On the Road,...,8784318.0,124.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The best teacher is experience.,On the Road,0.0,5.606,785.0,R


In [15]:
# Load in the dataframe from project part 1 as basics:
yr_2013 = pd.read_csv('Data/final_tmdb_data_2013.csv.gz')

yr_2013.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0255820,0.0,NaN,NaN,2000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,323498.0,en,Return to Babylon,...,0.0,75.0,[],Released,NaN,Return to Babylon,0.0,5.800,22.0,NaN
2,tt0359950,0.0,/8hSjOHRY4OUEpqxszYbMdem8z9C.jpg,NaN,90000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 35, '...",https://www.20thcenturystudios.com/movies/the-...,116745.0,en,The Secret Life of Walter Mitty,...,188133322.0,114.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Stop dreaming. Start living.,The Secret Life of Walter Mitty,0.0,7.172,7064.0,PG
3,tt0409379,0.0,/3tjCnZ9GFLiuMjAHxXAm5Tc98W2.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",http://insecretmovie.com/,172226.0,en,In Secret,...,0.0,101.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,In Secret,0.0,6.213,197.0,R
4,tt0452183,0.0,NaN,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}]",NaN,439174.0,en,Blunt Movie,...,0.0,83.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,From the creators of ABSOLUTELY NOTHING before...,Blunt Movie,0.0,3.700,3.0,R


In [16]:
# Load in the dataframe from project part 1 as basics:
yr_2014 = pd.read_csv('Data/final_tmdb_data_2014.csv.gz')

yr_2014.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0329539,0.0,NaN,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}]",NaN,274341.0,en,Rice Girl,...,0.0,95.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"For Windy Yee, a big break in Hollywood can me...",Rice Girl,0.0,1.000,2.0,NaN
2,tt0365907,0.0,/isrro0soStk2tSWMsI50lPPhUsU.jpg,NaN,28000000.0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,169917.0,en,A Walk Among the Tombstones,...,58834384.0,114.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some people are afraid of all the wrong things,A Walk Among the Tombstones,0.0,6.316,2622.0,R
3,tt0403935,0.0,/romORaZvZqMRjTPgw0HrAZpVn9V.jpg,NaN,0.0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,308165.0,hi,एक्शन जैकसन,...,0.0,144.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,"Naa Commitment, Naa Appointment, Only Punishment!",Action Jackson,0.0,4.400,27.0,NaN
4,tt0424859,0.0,NaN,NaN,0.0,[],NaN,285294.0,en,The Ninth Cloud,...,0.0,93.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Even Reality can't stop Zena.,The Ninth Cloud,0.0,4.000,1.0,NaN


In [17]:
# Load in the dataframe from project part 1 as basics:
yr_2015 = pd.read_csv('Data/final_tmdb_data_2015.csv.gz')

yr_2015.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0369610,0.0,/dF6FjTZzRTENfB4R17HDN20jLT2.jpg,"{'id': 328, 'name': 'Jurassic Park Collection'...",150000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.jurassicworld.com/,135397.0,en,Jurassic World,...,1.671537e+09,124.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The park is open.,Jurassic World,0.0,6.682,19377.0,PG-13
2,tt0420293,0.0,/b0p0OPx1ZDZVc8iDB1kle0Cc2H9.jpg,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}]",http://www.stanfordprisonexperimentfilm.com/,308032.0,en,The Stanford Prison Experiment,...,6.435570e+05,122.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They were given 2 weeks. It lasted 6 days.,The Stanford Prison Experiment,0.0,6.874,872.0,R
3,tt0462335,0.0,/wESuRMxELPAwo56qFRcoyI4p20F.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",http://www.highrisefilm.com,254302.0,en,High-Rise,...,4.289074e+06,119.0,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,Leave the real world behind,High-Rise,0.0,5.644,1068.0,R
4,tt0478970,0.0,/1K3JmSNUN8OpjYsCjc0Hy0SYxAb.jpg,"{'id': 422834, 'name': 'Ant-Man Collection', '...",130000000.0,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",http://marvel.com/movies/movie/180/ant-man,102899.0,en,Ant-Man,...,5.193120e+08,117.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Heroes don't get any bigger.,Ant-Man,0.0,7.082,18702.0,PG-13


In [18]:
# Load in the dataframe from project part 1 as basics:
yr_2018 = pd.read_csv('Data/final_tmdb_data_2018.csv.gz')

yr_2018.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0069049,0.0,/wXUcSJG6dqNKgIRgqYqX98UA1wz.jpg,NaN,12000000.0,"[{'id': 18, 'name': 'Drama'}]",https://www.netflix.com/title/80085566,299782.0,en,The Other Side of the Wind,...,0.0,122.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,40 years in the making.,The Other Side of the Wind,0.0,6.717,187.0,R
2,tt0192528,0.0,/gkBxjhtVEqgxOiRwRfWw1j7ez6K.jpg,NaN,5000000.0,"[{'id': 18, 'name': 'Drama'}]",NaN,567662.0,en,Reverse Heaven,...,0.0,104.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Heaven's in trouble and there's one man who ca...,Heaven & Hell,0.0,6.000,7.0,NaN
3,tt0360556,0.0,/7oy4miyq4WYYy0xtX6lbNVPrEsr.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam...",https://www.hbo.com/movies/fahrenheit-451,401905.0,en,Fahrenheit 451,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Knowledge is a dangerous thing,Fahrenheit 451,0.0,5.415,781.0,PG-13
4,tt0365545,0.0,/ljTYcQ3pkzYF52Z8ev1Z1UThnPy.jpg,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",https://www.netflix.com/title/80189630,519035.0,en,Nappily Ever After,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Let yourself grow,Nappily Ever After,0.0,7.189,786.0,NaN


In [20]:
# Load in the dataframe from project part 1 as basics:
yr_2019 = pd.read_csv('Data/final_tmdb_data_2019.csv.gz')

yr_2019.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0385887,0.0,/xMHeU84wLQ9uH9acJ0oPgdii9wO.jpg,NaN,26000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",https://www.motherlessbrooklynfilm.com,504562.0,en,Motherless Brooklyn,...,18377736.0,145.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Motherless Brooklyn,0.0,6.703,1306.0,R
2,tt0437086,0.0,/8RKBHHRqOMOLh5qW3sS6TSFTd8h.jpg,NaN,170000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...",https://family.20thcenturystudios.com/movies/a...,399579.0,en,Alita: Battle Angel,...,404980543.0,122.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,An angel falls. A warrior rises.,Alita: Battle Angel,0.0,7.218,8430.0,PG-13
3,tt0441881,0.0,/7uacpbrJd9g9xSTLgfyg1NwkAiu.jpg,NaN,23934823.0,"[{'id': 10752, 'name': 'War'}, {'id': 28, 'nam...",http://dangerclosemovie.com,508664.0,en,Danger Close: The Battle of Long Tan,...,2078370.0,119.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Outnumbered. Outgunned. Never out of courage,Danger Close: The Battle of Long Tan,0.0,6.863,281.0,R
4,tt0448115,0.0,/o7JVhqMmrex1TPbmuxl8YXVlcfl.jpg,"{'id': 724848, 'name': 'Shazam! Collection', '...",80000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",http://www.shazammovie.com,287947.0,en,Shazam!,...,366080049.0,132.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Just say the word.,Shazam!,0.0,7.034,8813.0,PG-13


In [21]:
#Concatenate the data into 1 dataframe for the remainder of the analysis.
con_df = pd.concat([yr_2010, yr_2011, yr_2012, yr_2013, yr_2014, yr_2015, yr_2018, yr_2019], ignore_index=True)
con_df.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0312305,0.0,/lqUbt2cy2pnqvxKefbQAtxLS0WA.jpg,NaN,0.0,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",http://www.qqthemovie.com/,23738.0,en,Quantum Quest: A Cassini Space Odyssey,...,0.0,45.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Quantum Quest: A Cassini Space Odyssey,0.0,7.900,8.0,NaN
2,tt0326965,0.0,/xt2klJdKCVGXcoBGQrGfAS0aGDE.jpg,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",http://www.inmysleep.com,40048.0,en,In My Sleep,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Sleepwalking can be deadly.,In My Sleep,0.0,5.318,33.0,PG-13
3,tt0331312,0.0,NaN,NaN,0.0,[],NaN,214026.0,en,This Wretched Life,...,0.0,0.0,[],Released,NaN,This Wretched Life,0.0,5.000,1.0,NaN
4,tt0393049,0.0,/gc9FN5zohhzCt05RkejQIIPLtBl.jpg,NaN,300000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,324352.0,en,Anderson's Cross,...,0.0,98.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Sometimes the boy next door is more than the b...,Anderson's Cross,0.0,4.000,5.0,NaN


In [15]:
con_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2599 entries, 0 to 2598
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                2599 non-null   object 
 1   adult                  2597 non-null   float64
 2   backdrop_path          1441 non-null   object 
 3   belongs_to_collection  211 non-null    object 
 4   budget                 2597 non-null   float64
 5   genres                 2597 non-null   object 
 6   homepage               170 non-null    object 
 7   id                     2597 non-null   float64
 8   original_language      2597 non-null   object 
 9   original_title         2597 non-null   object 
 10  overview               2553 non-null   object 
 11  popularity             2597 non-null   float64
 12  poster_path            2355 non-null   object 
 13  production_companies   2597 non-null   object 
 14  production_countries   2597 non-null   object 
 15  rele